In [6]:
# Import necessary libraries
from qutip import basis, tensor, Qobj, qeye, sigmax, sigmaz, sigmap, sigmam, snot
import pandas as pan
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.quantum_info import partial_trace
from qiskit.circuit.library import UnitaryGate
from qiskit.result import marginal_counts
from math import pi, sqrt
import numpy as np
from itertools import product
import os
import matplotlib.pyplot as plt

# Set Matplotlib to use a serif font
plt.rcParams['font.family'] = 'serif'
from qiskit.circuit.library import HGate, XGate



In [7]:
# Importing necessary modules and classes from BatSim package
from BatSim.Physical import Unconditional, Daemonic  # Importing Unconditional and Daemonic from RyCnot. If you intend to use the Physical model in the code, you need to replace 'RyCnot' with 'Physical'.
from BatSim.Calibration import Data  # Importing Data from Calibration
from BatSim.BestQubits import Select  # Importing Select from BestQubits
from BatSim.Plot import data_plot  # Importing data_plot from Plot

# Importing necessary functions and constants
from math import pi, sqrt, exp  # Importing pi, sqrt, and exp functions/constants from math module

# Importing Qiskit Aer Simulator from qiskit_aer package
from qiskit_aer import AerSimulator

# Importing IBMProvider and QiskitRuntimeService from qiskit_ibm_provider and qiskit_ibm_runtime packages
from qiskit_ibm_provider import IBMProvider
from qiskit_ibm_runtime import QiskitRuntimeService


from qiskit_aer import AerSimulator

backend = AerSimulator()

In [8]:
Steps = 1
omega = 1
kappa = 1

Shots  = 10000

pa, pd, P01, P10 =0.0292, 0.0123, 0.004276743038684305, 0.9929036144483037


In [9]:

Battery = basis(2, 0)
Ancilla = basis(2, 0)

def initialize():
    """
    Initialize the joint state and density matrix.

    Returns:
    Qobj: Density matrix of the initialized state.
    """
    Joint_State = tensor(basis(2, 0), basis(2, 0))
    Density = Joint_State * Joint_State.dag()
    return Density

# Define projective measurements
P00 = 1 - P10
P11 = 1 - P01
P0 = P00 * basis(2, 0) * basis(2, 0).dag() + P01 * basis(2, 1) * basis(2, 1).dag()
P1 = P11 * basis(2, 1) * basis(2, 1).dag() + P10 * basis(2, 0) * basis(2, 0).dag()
PP = tensor(qeye(2), snot() * P0 * snot().dag())
PM = tensor(qeye(2), snot() * P1 * snot().dag())

# Define noise operators
Noise_Identity = tensor(qeye(2), qeye(2))
Noise_z = tensor(sigmaz(), qeye(2))
AP0 = tensor(Qobj([[1, 0], [0, sqrt(1 - pa)]]), qeye(2))
AP1 = tensor(Qobj([[0, sqrt(pa)], [0, 0]]), qeye(2))

# Define the Hamiltonian and the unitary gate for charging
H = omega * (tensor(sigmax(), qeye(2))) + kappa * (tensor(sigmam(), sigmap()) + tensor(sigmap(), sigmam()))
M_Unitary = (-1j * H).expm()

def steps_sample_space(steps):
    """
    Generate the sample space for the given number of steps.

    Parameters:
    steps (int): Number of steps.

    Returns:
    tuple: Sorted sample space and corresponding decimal trajectories.
    """
    outcomes = list(product([0, 1], repeat=steps))
    sample_space = [list(outcome) for outcome in outcomes]
    sample_space = [[1 if outcome == 1 else 0 for outcome in step] for step in sample_space]
    decimal_trajectories = [int(''.join(map(str, step)), 2) for step in sample_space]
    sorted_lists = sorted(zip(sample_space, decimal_trajectories), key=lambda x: x[1])
    sorted_sample_space, sorted_decimal_trajectories = zip(*sorted_lists)
    
    for items in sorted_sample_space:
        items.reverse()

    return sorted_sample_space, sorted_decimal_trajectories

def find_unitary(Rho):
    """
    Find the unitary matrix to diagonalize the density matrix.

    Parameters:
    Rho (Qobj): Density matrix to be diagonalized.

    Returns:
    Qobj: Unitary matrix.
    """
    E = Rho.eigenstates()
    S1 = E[1][0].full()
    S2 = E[1][1].full()
    Di = Qobj([[S1[0][0], S2[0][0]], [S1[1][0], S2[1][0]]]).dag()
    Diagonal_Density = Di * Rho * Di.dag()
    Test_Matrix = Diagonal_Density.full()

    if Test_Matrix[0][0] > Test_Matrix[1][1]:
        Unitary = Di
    else:
        Unitary = sigmax() * Di

    return Unitary

# Start the loop to save density matrix for each step
Passive = []
H = 0.5 * (qeye(2) - sigmaz())
j = 0

for step in range(Steps):
    Passive_Energy = 0
    data_list = []
    j += 1
    num = f'{omega}-{kappa}-{j}'
    Trajectories, decimal = steps_sample_space(j)

    for Trajectory in Trajectories:
        Density = initialize()
        E = 0

        for Measure_Type in Trajectory:
            Evolved = M_Unitary * Density * M_Unitary.dag()  # Charge and interact the qubit with a specific theta in each step
            Evolved = pd * Noise_Identity * Evolved * Noise_Identity.dag() + (1 - pd) * Noise_z * Evolved * Noise_z.dag()
            Evolved = AP0 * Evolved * AP0.dag() + AP1 * Evolved * AP1.dag()

            if Measure_Type == 0:
                Meas_Den_P = Evolved * PP
                Probability_P = Meas_Den_P.tr()
                Meas_Den_P = Meas_Den_P / Probability_P
                P_Den = Meas_Den_P.ptrace(0)
                Battery = P_Den
                Density = tensor(Battery, Ancilla * Ancilla.dag())
            else:
                Meas_Den_M = Evolved * PM
                Probability_M = Meas_Den_M.tr()
                Meas_Den_M = Meas_Den_M / Probability_M
                M_Den = Meas_Den_M.ptrace(0)
                Battery = M_Den
                Density = tensor(Battery, Ancilla * Ancilla.dag())

        Rho = Battery
        Unitary = find_unitary(Rho)
        Matrix = Unitary.full()
        a = Matrix[0][0]
        b = Matrix[1][1]
        c = Matrix[0][1]
        d = Matrix[1][0]
        G_Den = (Unitary * Battery * Unitary.dag()) * H
        E = G_Den.tr()
        Passive_Energy += E
        data_list.append({'a': a, 'b': b, 'c': c, 'd': d})

    # Convert the list of dictionaries to a DataFrame and save it to a CSV file
    df = pan.DataFrame(data_list)
    directory = "Unitaries"
    file = f'Output-Mixed-Passive-{num}.csv'
    csv_file_path= os.path.join(directory, file)

    df.to_csv(csv_file_path, index=False, header=False)
    Passive.append(Passive_Energy / 2**j)


C:\Users\Navid Elyasi\AppData\Local\Temp\ipykernel_2152\179228176.py:20: DeprecationWarning: Importing functions/classes of the qip submodule directly from the namespace qutip is deprecated. Please import them from the submodule instead, e.g.
from qutip.qip.operations import cnot
from qutip.qip.circuit import QubitCircuit

  PP = tensor(qeye(2), snot() * P0 * snot().dag())
C:\Users\Navid Elyasi\AppData\Local\Temp\ipykernel_2152\179228176.py:21: DeprecationWarning: Importing functions/classes of the qip submodule directly from the namespace qutip is deprecated. Please import them from the submodule instead, e.g.
from qutip.qip.operations import cnot
from qutip.qip.circuit import QubitCircuit

  PM = tensor(qeye(2), snot() * P1 * snot().dag())


In [10]:
from qiskit_aer import AerSimulator 
backend = AerSimulator().from_backend(backend)



num = f'{omega}.{kappa}'

# Define the Hamiltonian
H = omega * (tensor(sigmax(), qeye(2))) + kappa * (tensor(sigmam(), sigmap()) + tensor(sigmap(), sigmam()))

# Generate the Unitary gate for the system
M_Unitary = (-1j * H).expm()


circuits = []
j = 0

# Loop over the number of steps
for step in range(Steps):
    Passive_Energy = 0
    data_list = []
    j += 1
    #num = f'{omega}-{kappa}-{j}'
    #directory = "Unitaries"
    #file = f'Output-Mixed-Passive-{num}.csv'
    #csv_file_path= os.path.join(directory, file)

    # Circuit setup
    q0 = QuantumRegister(1, name='Battery')
    q1 = QuantumRegister(1, name='Auxiliary')
    creg = ClassicalRegister(j + 1)
    qc = QuantumCircuit(q0, q1, creg)

    # Create an empty list to save the measurement results
    result_ = []

    # Start the collisional model for an arbitrary number of steps
    for i in range(j):
        M_gate = UnitaryGate(M_Unitary, label=rf'$\hat{{V}}$' )
        qc.append(M_gate, [q1, q0])
        h_gate = HGate(label=r'$\hat{U}_{\sf Had}$')
        qc.append(h_gate, q1)
        qc.measure(q1, creg[i])
        #qc.barrier()
        x_gate = XGate(label=r'$\hat{X}$' )
        with qc.if_test((creg[i], 1)):
            qc.append(x_gate, q1)
        #qc.reset(q1)
        #qc.barrier()
    
    with qc.switch(creg) as case: 
        for i in range(2**j):
            with case(i):
                # Retrieve the values from the CSV file
                Values = 1, 1, 0, 0
                a = Values[0]
                b = Values[1]
                c = Values[2]
                d = Values[3]
                
                # Create the unitary matrix using the obtained values
                matrix = [[a, c], [d, b]]
                
                # Convert it into a gate
                V = UnitaryGate(matrix, label=rf'$\hat{{U}}_{{a_{{{step+1}}}}}$')

                # Apply the gate to the battery qubit
                qc.append(V, q0)

    qc.measure(q0, creg[j])


    #circuits.append(qc)
    
    
display(qc.draw(output = 'mpl',  scale=10, filename = f"Pic{step}", idle_wires = False))


AttributeError: 'NoneType' object has no attribute 'get_edges'

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Button, HBox, VBox, Output
import time
from IPython.display import display, clear_output

class IsingModel:
    def __init__(self, L, T):
        """
        Initialize the 2D Ising Model.
        
        Parameters:
        - L: int, size of the LxL grid
        - T: float, initial temperature
        """
        self.L = L
        self.T = T
        self.grid = np.random.choice([-1, 1], size=(L, L))
        self.running = False
        self.steps = 0

    def calculate_energy(self):
        """
        Calculate the total energy of the system.
        """
        energy = 0
        for i in range(self.L):
            for j in range(self.L):
                spin = self.grid[i, j]
                neighbors = self.grid[(i+1) % self.L, j] + self.grid[i, (j+1) % self.L]
                energy -= spin * neighbors
        return energy

    def calculate_magnetization(self):
        """
        Calculate the magnetization of the system.
        """
        return np.sum(self.grid)

    def delta_energy(self, i, j):
        """
        Calculate the energy change if the spin at (i, j) is flipped.
        """
        spin = self.grid[i, j]
        neighbors = (
            self.grid[(i+1) % self.L, j] +
            self.grid[(i-1) % self.L, j] +
            self.grid[i, (j+1) % self.L] +
            self.grid[i, (j-1) % self.L]
        )
        return 2 * spin * neighbors

    def metropolis_step(self):
        """
        Perform a single Metropolis step to update the grid.
        """
        for _ in range(self.L ** 2):
            i, j = np.random.randint(0, self.L), np.random.randint(0, self.L)
            dE = self.delta_energy(i, j)
            if dE < 0 or np.random.rand() < np.exp(-dE / self.T):
                self.grid[i, j] *= -1

# Create the model
L = 20
T_initial = 2.0
ising_model = IsingModel(L, T_initial)

# Output widgets for the plots and lattice
output_plot = Output()
output_lattice = Output()

# Plot and interactive controls
temp_slider = FloatSlider(value=T_initial, min=0.1, max=5.0, step=0.1, description='Temperature')
start_button = Button(description="Start")
stop_button = Button(description="Stop")
resume_button = Button(description="Resume")

# Real-time variables for plotting
energies = []
magnetizations = []

def update_plot():
    """
    Update the energy and magnetization plots.
    """
    with output_plot:
        clear_output(wait=True)
        plt.figure(figsize=(10, 5))

        # Plot magnetization
        plt.subplot(2, 1, 1)
        plt.plot(magnetizations, label='Magnetization')
        plt.ylabel("Magnetization")
        plt.legend()

        # Plot energy
        plt.subplot(2, 1, 2)
        plt.plot(energies, label='Energy')
        plt.xlabel("Steps")
        plt.ylabel("Energy")
        plt.legend()

        plt.tight_layout()
        plt.show()

def update_lattice():
    """
    Update the lattice visualization.
    """
    with output_lattice:
        clear_output(wait=True)
        plt.imshow(ising_model.grid, cmap='coolwarm')
        plt.title("2D Ising Model Lattice")
        plt.colorbar(label="Spin")
        plt.show()

def simulate(change=None):
    """
    Run the simulation, updating plots and lattice.
    """
    while ising_model.running:
        ising_model.metropolis_step()
        
        # Record data for plots
        energy = ising_model.calculate_energy()
        magnetization = ising_model.calculate_magnetization()
        energies.append(energy)
        magnetizations.append(magnetization)

        # Update plots and lattice
        update_plot()
        update_lattice()
        
        # Delay to slow down the updates for better visualization
        time.sleep(0.1)

        # Increment steps
        ising_model.steps += 1

def start_simulation(b):
    """
    Start button callback: initialize simulation.
    """
    ising_model.running = True
    simulate()

def stop_simulation(b):
    """
    Stop button callback: pauses the simulation.
    """
    ising_model.running = False

def resume_simulation(b):
    """
    Resume button callback: continue from last step.
    """
    ising_model.running = True
    simulate()

def update_temperature(change):
    """
    Update the temperature when the slider changes.
    """
    ising_model.T = temp_slider.value

# Connect widgets to functions
start_button.on_click(start_simulation)
stop_button.on_click(stop_simulation)
resume_button.on_click(resume_simulation)
temp_slider.observe(update_temperature, names='value')

# Display the layout
display(
    HBox([
        VBox([temp_slider, start_button, stop_button, resume_button]), 
        output_lattice
    ]),
    output_plot
)



[notice] A new release of pip available: 22.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



     ------------------------------------ 139.8/139.8 kB 394.9 kB/s eta 0:00:00
     ---------------------------------------- 2.3/2.3 MB 979.2 kB/s eta 0:00:00
     -------------------------------------- 214.4/214.4 kB 4.3 MB/s eta 0:00:00
